## Introduction



In our last project we used data to estimate systems of food demand
using different datasets.  An output from that project was as set of
`cfe.Result` objects; these bundle together both data and the results
from the demand system estimation, and can be used for prediction as
well.  

Here we&rsquo;ll explore some of the uses of the `cfe.Result` class, using
an instance created using data from the Indian ICRISAT data.

If you don&rsquo;t already have the latest version of the `cfe` package
installed, grab it, along with some dependencies:



In [1]:
!pip install CFEDemands --upgrade --pre
!pip install xarray --upgrade
!pip install h5netcdf

In [1]:
import cfe

cfe.Result?

An instance `r` of `cfe.Result` can be made persistent with
`r.to_dataset('my_result.ds')`, which saves the instance &ldquo;on disk&rdquo; in
[NetCDF format](https://en.wikipedia.org/wiki/NetCDF), and can be loaded using `cfe.from_dataset`.  We use
this method below to load data and demand system estimated from the
Indian ICRISAT data:



In [1]:
r = cfe.result.from_dataset('Indian ICRISAT_small.ds')

Our focus earlier was on estimation, but the result `r` already has
encoded in it the results of that estimation.

There are parameters $\alpha$:



In [1]:
r.get_alpha(as_df=True).dropna()

Elasticity parameters $\beta$:



In [1]:
r.get_beta(as_df=True).dropna()

And parameters related to household characteristics $\delta$:



In [1]:
r.delta.to_dataframe().unstack('k').dropna()

The triple of paramters $(\alpha,\beta,\delta)$ completely describes
the demand system and the corresponding utility function (over the
goods we observe).



## Demands



As mentioned above, we&rsquo;ve estimated the parameters of a Frischian
demand system (demands that depend on prices and the households
marginal utility of expenditures).  But we can *compute* the
corresponding Marshallian (depends on prices and budget) or Hicksian
(depends on prices and the level of utility) demands for this same
population, using the `cfe.Result.demands` method.  

Let&rsquo;s compute Marshallian demands.  Start with a choice of budget $x$
and prices.  We&rsquo;ll use the earliest prices, and focus on
the first market we&rsquo;re interested in.



In [1]:
t = r.firstround  # First round
m = r.coords['m'][0] # First market

x = r.get_predicted_expenditures().sum('i')
medianx = x.where(x>0).sel(t=t,m=m).median('j').values # Budget (median household)

Now compute expenditures on different items.  The object `r` already knows what the estimated
parameters are, and uses those automatically:



In [1]:
xbar=r.demands(medianx) # Expenditures given median budget
xbar

So we can compute expenditures, but these are in units of Rupees.
We can get into physical units (say kilograms) by dividing coconut
expenditures by the price per kg in a given time and place:



In [1]:
USE_GOOD = 'Coconut'
    
# Reference prices for kgs at time t in market m
ref_price = r.prices.sel(t=t,m=m,u='Kilograms')

r.demands(medianx)[USE_GOOD]/ref_price.loc[USE_GOOD].values

Now we can trace out demands for a household with median budget but
varying prices of coconuts (holding other prices fixed):



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Scales of prices:
# Scales of prices:
P = np.linspace(1,10,100).tolist()

plt.plot([r.demands(medianx,p={USE_GOOD:scale})[USE_GOOD]/ref_price.loc[USE_GOOD].values for scale in P],P)

plt.xlabel("%s in Kgs" % USE_GOOD)
plt.ylabel('Price Scale')

Let&rsquo;s trace out demands for different budgets:



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X = [medianx*s for s in [.25,.5,1.,2,4]]
for myx in X:
    plt.plot([r.demands(myx,p={USE_GOOD:scale})[USE_GOOD]/ref_price.loc[USE_GOOD].values for scale in P],P)

plt.xlabel("%s in Kgs" % USE_GOOD)
plt.ylabel('Price Scale')
plt.legend(X)